In [4]:
import requests
import json
import datetime
import time
import yaml

In [5]:
file_path="C:\koreainvestment-autotrade-main\portfolioResult.json"

with open(file_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)


code_list=[]
for stock in json_data['stock']:
    code=list(stock.values())[0]
    stock_code = "".join(code)
    code_list.append(stock_code)
    
print(code_list)

weight_list=[]
for stock in json_data['stock']:
    weight=list(stock.values())[1]
    stock_weight = "".join(str(weight))
    weight_list.append(stock_weight)
    
print(weight_list)



['028670', '019170', '011810', '035720']
['0.2415624124477316', '0.2675114079874779', '0.24997676397125754', '0.2409494155935329']


In [6]:
seed = input("원하는 seed값을 적으시오: ")
seed

원하는 seed값을 적으시오: 1000000


'1000000'

In [7]:
seed_list=[]
i=0    

for i in range(len(weight_list)):
    seed_list.append(float(weight_list[i])*float(seed))

for i in range(len(seed_list)):
    seed_list[i] = int(seed_list[i])

print(seed_list)  

[241562, 267511, 249976, 240949]


In [8]:
with open("C:\koreainvestment-autotrade-main\config.yaml", encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
    
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']

In [9]:
def send_message(msg):
    """디스코드 메세지 전송"""
    now = datetime.datetime.now()
    message = {"content": f"[{now.strftime('%Y-%m-%d %H:%M:%S')}] {str(msg)}"}
    requests.post(DISCORD_WEBHOOK_URL, data=message)
    print(message)

In [10]:
def get_access_token():
    """토큰 발급"""
    headers = {"content-type": "application/json"}
    body = {"grant_type": "client_credentials",
            "appkey": APP_KEY,
            "appsecret": APP_SECRET}
    PATH = "oauth2/tokenP"
    URL = f"{URL_BASE}/{PATH}"
    res = requests.post(URL, headers=headers, data=json.dumps(body))
    ACCESS_TOKEN = res.json()["access_token"]
    return ACCESS_TOKEN

In [11]:
def hashkey(datas):
    """암호화"""
    PATH = "uapi/hashkey"
    URL = f"{URL_BASE}/{PATH}"
    headers = {
        'content-Type': 'application/json',
        'appKey': APP_KEY,
        'appSecret': APP_SECRET,
    }
    res = requests.post(URL, headers=headers, data=json.dumps(datas))
    hashkey = res.json()["HASH"]
    return hashkey

In [12]:
def get_current_price(code):
    """현재가 조회"""
    PATH = "uapi/domestic-stock/v1/quotations/inquire-price"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type": "application/json",
               "authorization": f"Bearer {ACCESS_TOKEN}",
               "appKey": APP_KEY,
               "appSecret": APP_SECRET,
               "tr_id": "FHKST01010100"}
    params = {
        "fid_cond_mrkt_div_code": "J",
        "fid_input_iscd": code,
    }
    res = requests.get(URL, headers=headers, params=params)
    return int(res.json()['output']['stck_prpr'])

In [13]:
def get_stock_balance():
    """주식 잔고조회"""
    PATH = "uapi/domestic-stock/v1/trading/inquire-balance"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type": "application/json",
               "authorization": f"Bearer {ACCESS_TOKEN}",
               "appKey": APP_KEY,
               "appSecret": APP_SECRET,
               "tr_id": "TTTC8434R",
               "custtype": "P",
               }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "AFHR_FLPR_YN": "N",
        "OFL_YN": "",
        "INQR_DVSN": "02",
        "UNPR_DVSN": "01",
        "FUND_STTL_ICLD_YN": "N",
        "FNCG_AMT_AUTO_RDPT_YN": "N",
        "PRCS_DVSN": "01",
        "CTX_AREA_FK100": "",
        "CTX_AREA_NK100": ""
    }
    res = requests.get(URL, headers=headers, params=params)
    stock_list = res.json()['output1']
    evaluation = res.json()['output2']
    stock_dict = {}
    send_message(f"====주식 보유잔고====")
    for stock in stock_list:
        if int(stock['hldg_qty']) > 0:
            stock_dict[stock['pdno']] = stock['hldg_qty']
            send_message(
                f"{stock['prdt_name']}({stock['pdno']}): {stock['hldg_qty']}주")
            time.sleep(0.1)
    send_message(f"주식 평가 금액: {evaluation[0]['scts_evlu_amt']}원")
    time.sleep(0.1)
    send_message(f"평가 손익 합계: {evaluation[0]['evlu_pfls_smtl_amt']}원")
    time.sleep(0.1)
    send_message(f"총 평가 금액: {evaluation[0]['tot_evlu_amt']}원")
    time.sleep(0.1)
    send_message(f"=================")
    return stock_dict
    

In [14]:
def get_balance():
    """현금 잔고조회"""
    PATH = "uapi/domestic-stock/v1/trading/inquire-psbl-order"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type": "application/json",
               "authorization": f"Bearer {ACCESS_TOKEN}",
               "appKey": APP_KEY,
               "appSecret": APP_SECRET,
               "tr_id": "TTTC8908R",
               "custtype": "P",
               }
    params = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "PDNO": "005930",
        "ORD_UNPR": "65500",
        "ORD_DVSN": "01",
        "CMA_EVLU_AMT_ICLD_YN": "Y",
        "OVRS_ICLD_YN": "Y"
    }
    res = requests.get(URL, headers=headers, params=params)
    cash = res.json()['output']['ord_psbl_cash']
    send_message(f"주문 가능 현금 잔고: {cash}원")
    return int(cash)

In [15]:
def buy(code, qty):
    """주식 시장가 매수"""
    PATH = "uapi/domestic-stock/v1/trading/order-cash"
    URL = f"{URL_BASE}/{PATH}"
    data = {
        "CANO": CANO,
        "ACNT_PRDT_CD": ACNT_PRDT_CD,
        "PDNO": code,
        "ORD_DVSN": "01",
        "ORD_QTY": str(int(qty)),
        "ORD_UNPR": "0",
    }
    headers = {"Content-Type": "application/json",
               "authorization": f"Bearer {ACCESS_TOKEN}",
               "appKey": APP_KEY,
               "appSecret": APP_SECRET,
               "tr_id":"VTTC0802U",
               "custtype": "P",
               "hashkey": hashkey(data)
               }
    res = requests.post(URL, headers=headers, data=json.dumps(data))
    if res.json()['rt_cd'] == '0':
        send_message(f"[매수 성공]{str(res.json())}")
        return True
    else:
        send_message(f"[매수 실패]{str(res.json())}")
        return False

In [20]:
# 자동매매 시작
try:
    #API 사용을 위해 발급받는 토큰
    ACCESS_TOKEN = get_access_token()
    
    bought_list = [] # 매수 완료된 종목 리스트

    target_buy_count = 10 
    buy_percent = 0.33
   
    send_message(f"===국내 주식 자동매매 프로그램을 시작합니다===")
    
    t_now = datetime.datetime.now()
    t_start = t_now.replace(hour=9, minute=0, second=0, microsecond=0)
    t_exit = t_now.replace(hour=16, minute=30, second=0,microsecond=0)
    today = datetime.datetime.today().weekday()
    
    if today == 5 or today == 6:  # 토요일이나 일요일이면 프로그램 자동 종료
        send_message("주말이므로 매매가 불가능합니다.")
    if t_start < t_now < t_exit :
        for code in code_list:
            for i in range(len(code_list)):
                if len(bought_list) < target_buy_count:
                    buy_qty = 0
                    current_price = get_current_price(code)# 현재 가격
                    buy_qty = int(seed_list[i] // current_price)
                    if buy_qty > 0:
                            send_message(f"현재가:{current_price}원의 종목 코드:{code}를 매수를 시도합니다.")
                            result = buy(code, buy_qty)
                            if result:
                                soldout = False
                    time.sleep(1)
        time.sleep(1)
    if (t_exit < t_now)|(t_start > t_now):  # PM 03:30 ~ :프로그램 종료
        send_message("주식 장이 마감되었습니다.")    

except Exception as e:
    send_message(f"오류발생.", e)
    time.sleep(1)

{'content': '[2022-11-16 15:51:40] ===국내 주식 자동매매 프로그램을 시작합니다==='}
{'content': '[2022-11-16 15:51:41] 현재가:5370원의 종목 코드:028670를 매수를 시도합니다.'}
{'content': "[2022-11-16 15:51:42] [매수 실패]{'rt_cd': '1', 'msg_cd': '40580000', 'msg1': '모의투자 장종료 입니다.'}"}
{'content': '[2022-11-16 15:51:43] 현재가:5370원의 종목 코드:028670를 매수를 시도합니다.'}
{'content': "[2022-11-16 15:51:44] [매수 실패]{'rt_cd': '1', 'msg_cd': '40580000', 'msg1': '모의투자 장종료 입니다.'}"}
{'content': '[2022-11-16 15:51:45] 현재가:5370원의 종목 코드:028670를 매수를 시도합니다.'}
{'content': "[2022-11-16 15:51:46] [매수 실패]{'rt_cd': '1', 'msg_cd': '40580000', 'msg1': '모의투자 장종료 입니다.'}"}
{'content': '[2022-11-16 15:51:48] 현재가:5370원의 종목 코드:028670를 매수를 시도합니다.'}
{'content': "[2022-11-16 15:51:49] [매수 실패]{'rt_cd': '1', 'msg_cd': '40580000', 'msg1': '모의투자 장종료 입니다.'}"}
{'content': '[2022-11-16 15:51:50] 현재가:28850원의 종목 코드:019170를 매수를 시도합니다.'}
{'content': "[2022-11-16 15:51:51] [매수 실패]{'rt_cd': '1', 'msg_cd': '40580000', 'msg1': '모의투자 장종료 입니다.'}"}
{'content': '[2022-11-16 15:51:53] 현재가

In [21]:
PAutoTrade_info = []

PAutoTrade_info.append(code_list) # 종목
PAutoTrade_info.append(seed) # 시드머니
PAutoTrade_info.append(buy_qty) #종목 당 매수 개수
PAutoTrade_info.append(current_price) # 현재가

print(PAutoTrade_info)

[['028670', '019170', '011810', '035720'], '1000000', 4, 58600]


In [22]:
PAT_dict = {"PAutoTrade":{"stk_cd":code_list, "seed":seed, "stk_buyN":buy_qty, "current_price":current_price}}
print(PAT_dict)

{'PAutoTrade': {'stk_cd': ['028670', '019170', '011810', '035720'], 'seed': '1000000', 'stk_buyN': 4, 'current_price': 58600}}


In [23]:
import json

PAT_json = json.dumps(PAT_dict, indent=4)
print(PAT_json)

{
    "PAutoTrade": {
        "stk_cd": [
            "028670",
            "019170",
            "011810",
            "035720"
        ],
        "seed": "1000000",
        "stk_buyN": 4,
        "current_price": 58600
    }
}
